In [1]:
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,East London,ZA,2020-08-29 20:14:07,-33.02,27.91,55.72,57,66,19.71
1,1,Kalmunai,LK,2020-08-29 20:14:12,7.42,81.82,81.03,76,64,5.75
2,2,Mahebourg,MU,2020-08-29 20:14:17,-20.41,57.70,71.60,73,75,14.99
3,3,Ribeira Grande,PT,2020-08-29 20:14:18,38.52,-28.70,77.00,88,75,10.29
4,4,Saint-Philippe,RE,2020-08-29 20:14:19,-21.36,55.77,71.60,68,41,13.87


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Kalmunai,LK,2020-08-29 20:14:12,7.42,81.82,81.03,76,64,5.75
3,3,Ribeira Grande,PT,2020-08-29 20:14:18,38.52,-28.70,77.00,88,75,10.29
6,6,Albany,US,2020-08-29 20:14:00,42.60,-73.97,80.01,82,58,9.37
7,7,Bengkulu,ID,2020-08-29 20:14:31,-3.80,102.27,76.19,84,89,3.71
8,8,Bethel,US,2020-08-29 20:10:57,41.37,-73.41,79.00,78,75,9.17
10,10,Biak,ID,2020-08-29 20:14:44,-0.91,122.88,76.08,81,100,8.48
12,12,Puerto Ayora,EC,2020-08-29 20:14:45,-0.74,-90.35,77.00,57,96,28.86
14,14,Najran,SA,2020-08-29 20:14:45,17.49,44.13,86.00,19,45,3.00
21,21,Hilo,US,2020-08-29 20:14:48,19.73,-155.09,82.40,58,1,3.36
24,24,Butaritari,KI,2020-08-29 20:14:50,3.07,172.79,82.96,69,69,10.87


In [10]:
preferred_cities_df.isnull().sum()
preferred_cities_df.count()

City_ID       190
City          190
Country       190
Date          190
Lat           190
Lng           190
Max Temp      190
Humidity      190
Cloudiness    190
Wind Speed    190
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kalmunai,LK,81.03,7.42,81.82,
3,Ribeira Grande,PT,77.00,38.52,-28.70,
6,Albany,US,80.01,42.60,-73.97,
7,Bengkulu,ID,76.19,-3.80,102.27,
8,Bethel,US,79.00,41.37,-73.41,
10,Biak,ID,76.08,-0.91,122.88,
12,Puerto Ayora,EC,77.00,-0.74,-90.35,
14,Najran,SA,86.00,17.49,44.13,
21,Hilo,US,82.40,19.73,-155.09,
24,Butaritari,KI,82.96,3.07,172.79,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Kalmunai,LK,81.03,7.42,81.82,VS Villa
3,Ribeira Grande,PT,77.00,38.52,-28.70,Quinta da Meia Eira
6,Albany,US,80.01,42.60,-73.97,
7,Bengkulu,ID,76.19,-3.80,102.27,Grage Hotel Bengkulu
8,Bethel,US,79.00,41.37,-73.41,Courtyard by Marriott Danbury
10,Biak,ID,76.08,-0.91,122.88,Kost Ariz
12,Puerto Ayora,EC,77.00,-0.74,-90.35,Finch Bay Galapagos Hotel
14,Najran,SA,86.00,17.49,44.13,محمد هادي ال فطيح
21,Hilo,US,82.40,19.73,-155.09,Hilo Hawaiian Hotel
24,Butaritari,KI,82.96,3.07,172.79,Isles Sunset Lodge


In [13]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [14]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))